In [226]:
from ase import Atoms
from ase.io import write,read
import re 

def open_gaussian_out(gauss_out_path):
    with open(gauss_out_path) as gauss_out:
        try:
            gauss_out = gauss_out.readlines()
        except UnicodeDecodeError:
            return False
        
    return gauss_out

def get_atoms_symbols_and_positions(gauss_out):
    symbols = []
    positions = []
    reg = " [OCNH],0,([0-9]*\.[0-9]+(,[0-9]*\.[0-9]+)+)"
    reg = ' [OCNH],(([+-]?(?=\.\d|\d)(?:\d+)?(?:\.?\d*))(?:[eE]([+-]?\d+))?(,([+-]?(?=\.\d|\d)(?:\d+)?(?:\.?\d*))(?:[eE]([+-]?\d+))?)+)'
    for line in gauss_out:
        if not len(line) > 0:
            continue
        if re.search(reg, line) and len(line.split(',')) == 5:  
            symbol, _, x, y ,z = line.split(',')
            symbols.append(symbol.replace(' ', ''))
            positions.append((x, y ,z))
    return symbols, positions 

def get_chemical_shift(gauss_out):
    prefactors = {'C' : 161.5794+24.637, 'N': 213.8677+70, 'H' : 31.78218333+0, 'O' : 0}
    chemical_shifts = []

    reg = "Anisotropy"
    for line in gauss_out:
        if not len(line) > 0:
            continue
        if re.search(reg, line):
            _, symbol, _,_, anisotropy, _,_,_ = line.split()
            if symbol == 'O' or symbol == 'Si':
                chemical_shifts.append(0)
            else:
                chemical_shifts.append(prefactors[symbol] - float(anisotropy))
    return chemical_shifts
    
def get_atoms_object(symbols, positions, chemical_shifts):
    if len(symbols) == len(chemical_shifts):
        return Atoms(symbols=symbols, positions=positions,charges=chemical_shifts)
    else:
        return False



In [227]:
def write_xyz_chemical_shift(gauss_out_path,name_file):
    try:
        atoms = read(gauss_out_path)
        gauss_out = open_gaussian_out(gauss_out_path)
        chemical_shifts = get_chemical_shift(gauss_out)
        if len(chemical_shifts) == 0:
            print(f'ERROR: no chemical shifts in {name_file}')
            return False
        atoms.set_initial_charges(chemical_shifts)
        atoms.center()
        write(f'{atoms.get_chemical_formula()}_{name_file}.xyz', atoms)
        
    except StopIteration:
        print('Reading input... ')
        gauss_out = open_gaussian_out(gauss_out_path)
        if not gauss_out:
            print('Input is bad')
            return False

        print('Calculating Chemical Shifts...')
        chemical_shifts = get_chemical_shift(gauss_out)
        if len(chemical_shifts) == 0:
            print(f'ERROR: no chemical shifts in {name_file}')
            return False

        print('Extracting atoms coordinates...')
        symbols, positions = get_atoms_symbols_and_positions(gauss_out)
        if len(symbols) == 0:
            print('ERROR: did not find atomic coordinates in the file')
            return False
            
        print('Creating Molecule...')
        atoms = get_atoms_object(symbols, positions,chemical_shifts)
        if atoms:
            atoms.center()
            print('Writing xyz file...')
            write(f'{atoms.get_chemical_formula()}_{name_file}.xyz', atoms)
        else:
            print('Could not write xyz file')

In [228]:
import os

exs = ['PBE0', 'PBE1PBE', 'B3LYP', 'M062X']

for dirname, dirs, files in os.walk('/media/riccardo/UNICA/Lysine/'):
    for file in files:
        gauss_out_name, extension = os.path.splitext(file)
        if extension == '.log':
            gauss_out_path = os.path.join(dirname,file)
            exchange = 'PBE0'
            for ex in exs:
                if ex in gauss_out_path.split('/'):
                    exchange = ex
            if exchange == 'PBE1PBE':
                exchange = 'PBE0'
            gauss_out_name = f'{exchange}_{gauss_out_name}'
            write_xyz_chemical_shift(gauss_out_path, gauss_out_name)

Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_trimerino
Reading input... 
Calculating Chemical Shifts...
Extracting atoms coordinates...
Creating Molecule...
Writing xyz file...
Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_trimerino_pro
Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_nmr_trimerino
Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_trimerinoFINALE
Reading input... 
Calculating Chemical Shifts...
Extracting atoms coordinates...
Creating Molecule...
Writing xyz file...
Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_pentamer-frame-25
Reading input... 
Calculating Chemical Shifts...
Extracting atoms coordinates...
ERROR: did not find atomic coordinates in the file
Reading input... 
Calculating Chemical Shifts...
ERROR: no chemical shifts in PBE0_tetramer-frame-17
Reading input... 
Calculating Chemic

['/media/riccardo/UNICA/Lysine/Dimer/PBE1PBE/nmr/TH_4HB_3O_2N/nmr_dir_mol1/nmr_mol1.log']